#  Drawing a brain with Bokeh

### Loading the info
##### MRI Image --> Anlysis of different brain areas --> Huge matrix of connections

To know the length that each area should have we need first of all to know the number of connections that each area has.

In [2]:
import numpy as np
connectome = np.load('connectome.npy')[:50,:50]
weights_of_areas = (connectome.sum(axis=0) + connectome.sum(axis=1)) - connectome.diagonal()

Now that we know the number of connections per area, I can know the length that it would represent in a circunference. For that, I'll use radians and **_radius=1_** to make operations easier.

In [3]:
from math import pi
areas_in_radians = (weights_of_areas/weights_of_areas.sum())*2*pi

If this is this operation was correct, the sum should be near a 2π radians.

In [4]:
from math import isclose
isclose(2*pi, areas_in_radians.sum())

True

Cool! An arc of a circle is a "portion" of the circumference of the circle. The length of an arc is simply the length of its "portion" of the circumference. Now that we now the length of each area in a circunference we need to know the start and end point of the arc in order to be able to draw them. The radian measure, $\emptyset$, of a central angle of a circle is defined as the ratio of the length of the arc the angle subtends, $s$, divided by the radius of the circle, $r$. 

$$\emptyset = \frac{s}{r} = \frac{lenght\ of\ arc}{length\ of\ radius} = \frac{s}{1}$$

So in this case $\emptyset = s$ makes everything much easier. Our points must start on zero, so I add a zero in the begining of the array.

In [5]:
points = np.zeros((areas_in_radians.shape[0]+1)) # We add a zero in the begging for the cumsum
points[1:] = areas_in_radians
points = points.cumsum()

In [6]:
start_angle = points[:-1]
end_angle = points[1:]

Let's add some color using the function we already created for the previous connectome.

In [7]:
from colorsys import hsv_to_rgb
def gen_color(h):
    golden_ratio = (1 + 5 ** 0.5) / 2
    h += golden_ratio
    h %= 1
    return '#{:02X}{:02X}{:02X}'.format(*tuple(int(a*100) for a in hsv_to_rgb(h, 0.55, 2.3)))

In [8]:
colors = np.array([gen_color(area/areas_in_radians.shape[0]) for area in range(areas_in_radians.shape[0])])

In [9]:
colors[0]

'#678CE5'

Creating the arcs

In [10]:
from bokeh.plotting import output_notebook, show, figure, ColumnDataSource
from bokeh.models import Range1d

In [11]:
arcs = ColumnDataSource(
    data=dict(
        x=np.zeros(connectome.shape[0]),
        y=np.zeros(connectome.shape[0]),
        start=start_angle,
        end=end_angle,
        colors=colors)
    )

In [12]:
p = figure(outline_line_color="white",
           toolbar_location="below")

# grid configurations
p.x_range = Range1d(-1.1, 1.1)
p.y_range = Range1d(-1.1, 1.1)
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.arc(x='x', y='y', start_angle='start', end_angle='end',
      radius=1, line_color='colors', source=arcs, line_width=10)

In [13]:
output_notebook()
show(p)

Loading BokehJS ...

## I got those arcs, now what ? → *bokeh.crazy_scientific_mode = True*
Now let's get into the real stuff. Drawing so many lines is not an easy task, so I did it using OOP to make it more understandable. I'm sure there is someone reading this and thinking, *"meh, I can do that without declaring a class"* for that people; please, do it and share your approach.

In [14]:
from math import cos, sin
class Area():
    """
    It represents a brain area. It will create a list of available points throught the arc representing that area and then we will
    use those points as start and end for the beziers.
    """
    def __init__(self, n_conn, start_point, end_point):
        self.n_conn = n_conn # Number of connections in that area
        self.start_point = start_point # The start point of the arc representing the area
        self.end_point = end_point 
        free_points_angles = np.linspace(start_point, end_point, n_conn) # Equally spaced points between start point and end point 
        self.free_points = [[cos(angle), sin(angle)] for angle in free_points_angles] # A list of available X,Y to consume

Now I generate each of the regions in the connectome as an object I store in a list.

In [15]:
all_areas = []
for i in range(start_angle.shape[0]):
    all_areas.append(Area(weights_of_areas[i], start_angle[i], end_angle[i]))

Each of those areas we just created has all the points(x,y) wich the connections will start from or go. Those points are in the same space of arcs we defined earlier. That will create the illusion that the lines are created from the arc, as they will share the same color that the arc they were generated from.

In [16]:
all_connections = []
for j, region1 in enumerate(connectome):
    # Get the connections origin region
    region_a = all_areas[j]
    color = colors[j]
    weight = weights_of_areas[j]
    
    for k, n_connections_region2 in enumerate(region1):
        # Get the connection destination region
        region_b = all_areas[k]
        for i in range(int(n_connections_region2)):
            p1 = region_a.free_points.pop()
            p2 = region_b.free_points.pop()
            # Get both regions free points and create a connection with the data
            all_connections.append(p1 + p2 + [color, weight])

In [17]:
len(all_connections)

14050

We have `4740` connections in total that we will draw in the plot.

In [18]:
import pandas as pd
connections_df = pd.DataFrame(all_connections, dtype=str)
connections_df.columns = ["start_x","start_y","end_x","end_y","colors", "weight"]

I store the values in the DataFrame as strings to store as many precission as possible. I found that storing them as floats, made me lose lot of precission and bokeh is taking the string values without problems.

Now we create the curves and add them to the Dataframe

In [19]:
connections_df["cx0"] = connections_df.start_x.astype("float64")/2
connections_df["cy0"] = connections_df.start_y.astype("float64")/2
connections_df["cx1"] = connections_df.end_x.astype("float64")/2
connections_df["cy1"] = connections_df.end_y.astype("float64")/2

We standarize the weights and give them a value that will make them visible. With this, the most important areas will be much opaque than those with less representation.

In [20]:
connections_df.weight = (connections_df.weight.astype("float64")/connections_df.weight.astype("float64").sum())*1000

In [21]:
connections_df.head()

,start_x,start_y,end_x,end_y,colors,weight,cx0,cy0,cx1,cy1
0,0.999995,0.00313041,0.765501,0.643434,#677DE5,0.000337,0.499998,0.001565,0.382751,0.321717
1,0.999996,0.00288961,0.765645,0.643263,#677DE5,0.000337,0.499998,0.001445,0.382823,0.321632
2,0.999996,0.00264881,-0.591938,0.805983,#677DE5,0.000337,0.499998,0.001324,-0.295969,0.402992
3,0.999997,0.00240801,-0.591758,0.806116,#677DE5,0.000337,0.499999,0.001204,-0.295879,0.403058
4,0.999998,0.00216721,-0.591578,0.806248,#677DE5,0.000337,0.499999,0.001084,-0.295789,0.403124


Once again, using ColumnDataSource I store all the data from the pandas DataFrame in a bokeh compatible object 

In [22]:
# Bezier lines
beziers = ColumnDataSource(connections_df)

Aaaaaaand let's plot it

In [23]:
p2 = figure(title="Connectomme",
           outline_line_color="white",
           toolbar_location="below")

# grid configurations
p2.x_range = Range1d(-1.1, 1.1)
p2.y_range = Range1d(-1.1, 1.1)
p2.xaxis.visible = False
p2.yaxis.visible = False
p2.xgrid.grid_line_color = None
p2.ygrid.grid_line_color = None

The glyphs...

In [25]:
p2.bezier('start_x', 'start_y', 'end_x', 'end_y', 'cx0', 'cy0', 'cx1', 'cy1',
             source=beziers,
             line_alpha='weight',
             line_color='colors')
p2.arc(x='x', y='y', start_angle='start', end_angle='end',
      radius=1, line_color='colors', source=arcs, line_width=10)

Voilá !

In [26]:
output_notebook()
show(p2)

Loading BokehJS ...

## Cool, but some things are still missing...
This is a work in progress, and by working in this visualization I found several frontiers in Bokeh. Things that aren't implemented yet but I hope that between my good intentions and the help of the community we can get working. That the you will be able to make Chord graphs just by passing an square matrix.

Things to be done:
- Add WebGL support to arcs and beziers
- Add HoverTool to arcs to display information
- Add TapTool to arcs to be able to display only the connections originated in one of the areas
- Create an interface to this

My hope is that in the next several months we can see this intigrated inside Bokeh.
Thanks for passing by !
